In [ ]:
"""
Transaction-Level Data Summary Generator
=========================================
Reads a CSV file with ~87,000 transaction rows and produces a
richly formatted multi-sheet Excel summary workbook.
"""

import pandas as pd
import numpy as np
from openpyxl import Workbook
from openpyxl.styles import Font, PatternFill, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from pathlib import Path

# Get current working directory (EDA folder)
BASE_DIR = Path.cwd()

# Go one level up
PROJECT_ROOT = BASE_DIR.parent

INPUT_PATH = PROJECT_ROOT / "outputs" / "transactions.csv"
OUTPUT_PATH = PROJECT_ROOT /"EDA"/ "transactions_summary.xlsx"

print(PROJECT_ROOT)

c:\Users\VISHNUPRIYA\OneDrive\Desktop\Freelancing\AIGEN\smartsentry_aml_model


In [3]:


print(f"📂 Loading data from: {INPUT_PATH}")
df = pd.read_csv(INPUT_PATH, parse_dates=["timestamp", "date"], low_memory=False)
print(f"✅ Loaded {len(df):,} rows × {df.shape[1]} columns")

TOTAL = len(df)


📂 Loading data from: c:\Users\VISHNUPRIYA\OneDrive\Desktop\Freelancing\AIGEN\smartsentry_aml_model\outputs\transactions.csv
✅ Loaded 86,992 rows × 109 columns


In [4]:

# ── Helper functions ────────────────────────────────────────────────────────────

def pct_df(series, label):
    d = series.value_counts().reset_index()
    d.columns = [label, "Count"]
    d["Percentage (%)"] = (d["Count"] / TOTAL * 100).round(2)
    return d

def stats_df(series, label):
    d = series.describe().round(4).reset_index()
    d.columns = ["Statistic", label]
    return d

def flag_summary(series, label, mapping=None):
    d = series.value_counts().reset_index()
    d.columns = [label, "Count"]
    if mapping:
        d[label] = d[label].map(mapping)
    d["Percentage (%)"] = (d["Count"] / TOTAL * 100).round(2)
    return d

def rule_summary(rule_cols, df):
    rows = []
    for col in rule_cols:
        fired = int(df[col].sum())
        rows.append({
            "Rule Name": col.replace("rule_", "").replace("_", " ").title(),
            "Column": col,
            "Times Fired": fired,
            "Percentage (%)": round(fired / TOTAL * 100, 2),
        })
    return pd.DataFrame(rows).sort_values("Times Fired", ascending=False).reset_index(drop=True)

# ── Styling constants ──────────────────────────────────────────────────────────

HDR_FILL  = PatternFill("solid", start_color="1F3864")
SUB_FILL  = PatternFill("solid", start_color="2E75B6")
HDR2_FILL = PatternFill("solid", start_color="BDD7EE")
ALT_FILL  = PatternFill("solid", start_color="D9E1F2")
WHITE     = PatternFill("solid", start_color="FFFFFF")
RED_FILL  = PatternFill("solid", start_color="FFE0E0")
THIN = Border(
    left=Side(style="thin", color="B8CCE4"),
    right=Side(style="thin", color="B8CCE4"),
    top=Side(style="thin", color="B8CCE4"),
    bottom=Side(style="thin", color="B8CCE4"),
)

def hdr(ws, row, col, val, size=11, fill=HDR_FILL, color="FFFFFF"):
    c = ws.cell(row=row, column=col, value=val)
    c.font = Font(name="Arial", bold=True, size=size, color=color)
    c.fill = fill
    c.alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
    c.border = THIN
    return c

def dat(ws, row, col, val, alt=False, bold=False, center=False, red=False):
    c = ws.cell(row=row, column=col, value=val)
    c.font = Font(name="Arial", size=10, bold=bold, color="1F2D3D")
    c.fill = RED_FILL if red else (ALT_FILL if alt else WHITE)
    c.alignment = Alignment(horizontal="center" if center else "left", vertical="center")
    c.border = THIN
    return c

def write_block(ws, sr, sc, title, dataframe, flag_high=None):
    """
    Write a titled table block at (sr, sc).
    flag_high: column name where value > threshold triggers red fill.
    Returns the next available row.
    """
    ec = sc + len(dataframe.columns) - 1
    ws.merge_cells(start_row=sr, start_column=sc, end_row=sr, end_column=ec)
    hdr(ws, sr, sc, title, size=10, fill=SUB_FILL)
    ws.row_dimensions[sr].height = 22

    for ci, col_name in enumerate(dataframe.columns, start=sc):
        hdr(ws, sr + 1, ci, col_name, size=9, fill=HDR2_FILL, color="1F2D3D")

    for ri, (_, row_data) in enumerate(dataframe.iterrows()):
        alt = ri % 2 == 1
        for ci, val in enumerate(row_data, start=sc):
            is_red = False
            if flag_high and dataframe.columns[ci - sc] == "Percentage (%)" and isinstance(val, float) and val > flag_high:
                is_red = True
            dat(ws, sr + 2 + ri, ci, val, alt=alt, center=(ci > sc), red=is_red)

    # Auto column widths
    for ci in range(sc, ec + 1):
        cl = get_column_letter(ci)
        max_w = max(len(str(ws.cell(r, ci).value or "")) for r in range(sr, sr + 2 + len(dataframe)))
        ws.column_dimensions[cl].width = min(max(max_w + 4, 14), 42)

    return sr + 2 + len(dataframe) + 2

# ══════════════════════════════════════════════════════════════════════════════
# BUILD SUMMARIES
# ══════════════════════════════════════════════════════════════════════════════

# ── 1. Transaction Overview ──────────────────────────────────────────────────
fraud_count    = int(df["label"].sum()) if "label" in df.columns else 0
legit_count    = TOTAL - fraud_count
fraud_pct      = round(fraud_count / TOTAL * 100, 2)
total_amount   = df["amount"].sum()
avg_amount     = df["amount"].mean()
median_amount  = df["amount"].median()
max_amount     = df["amount"].max()

overview_kpis = pd.DataFrame({
    "KPI": [
        "Total Transactions", "Fraud Transactions", "Legitimate Transactions",
        "Fraud Rate (%)", "Total Amount (INR)", "Avg Amount (INR)",
        "Median Amount (INR)", "Max Amount (INR)",
        "Unique Customers", "Unique Accounts (Sender)",
        "Unique Devices", "Unique Beneficiaries",
        "Date Range Start", "Date Range End",
    ],
    "Value": [
        f"{TOTAL:,}", f"{fraud_count:,}", f"{legit_count:,}",
        f"{fraud_pct}%", f"₹{total_amount:,.2f}", f"₹{avg_amount:,.2f}",
        f"₹{median_amount:,.2f}", f"₹{max_amount:,.2f}",
        df["customer_id"].nunique() if "customer_id" in df.columns else "N/A",
        df["sender_account_id"].nunique() if "sender_account_id" in df.columns else "N/A",
        df["device_id"].nunique() if "device_id" in df.columns else "N/A",
        df["beneficiary_id"].nunique() if "beneficiary_id" in df.columns else "N/A",
        str(df["timestamp"].min())[:10] if "timestamp" in df.columns else "N/A",
        str(df["timestamp"].max())[:10] if "timestamp" in df.columns else "N/A",
    ]
})

# ── 2. Fraud Type breakdown ──────────────────────────────────────────────────
fraud_type_df = pct_df(df["fraud_type"], "Fraud Type") if "fraud_type" in df.columns else pd.DataFrame()

# ── 3. Transaction characteristics ───────────────────────────────────────────
channel_df     = pct_df(df["channel"], "Channel") if "channel" in df.columns else pd.DataFrame()
txn_type_df    = pct_df(df["transaction_type"], "Transaction Type") if "transaction_type" in df.columns else pd.DataFrame()
debit_df       = pct_df(df["debit_credit"], "Debit/Credit") if "debit_credit" in df.columns else pd.DataFrame()
cash_df        = flag_summary(df["cash_flag"], "Cash Flag", {0: "Non-Cash", 1: "Cash"}) if "cash_flag" in df.columns else pd.DataFrame()
amount_stats   = stats_df(df["amount"], "Amount (INR)") if "amount" in df.columns else pd.DataFrame()

# ── 4. Temporal distribution ──────────────────────────────────────────────────
hour_df        = pct_df(df["hour"], "Hour of Day") if "hour" in df.columns else pd.DataFrame()
dow_map        = {0:"Monday",1:"Tuesday",2:"Wednesday",3:"Thursday",4:"Friday",5:"Saturday",6:"Sunday"}
dow_df         = pct_df(df["day_of_week"].map(dow_map) if "day_of_week" in df.columns else df["day_of_week"], "Day of Week") if "day_of_week" in df.columns else pd.DataFrame()
month_df       = pct_df(df["month"], "Month") if "month" in df.columns else pd.DataFrame()
night_df       = flag_summary(df["is_night"], "Is Night", {0:"Day",1:"Night"}) if "is_night" in df.columns else pd.DataFrame()
weekend_df     = flag_summary(df["is_weekend"], "Is Weekend", {0:"Weekday",1:"Weekend"}) if "is_weekend" in df.columns else pd.DataFrame()
biz_df         = flag_summary(df["is_business_hours"], "Business Hours", {0:"Non-Business",1:"Business"}) if "is_business_hours" in df.columns else pd.DataFrame()

# ── 5. Customer & Account Profile ─────────────────────────────────────────────
occ_df         = pct_df(df["occupation"], "Occupation") if "occupation" in df.columns else pd.DataFrame()
kyc_df         = pct_df(df["kyc_level"], "KYC Level") if "kyc_level" in df.columns else pd.DataFrame()
crisk_df       = pct_df(df["country_risk"], "Country Risk") if "country_risk" in df.columns else pd.DataFrame()
inc_df         = pct_df(df["income_bracket"], "Income Bracket") if "income_bracket" in df.columns else pd.DataFrame()
cust_risk_df   = pct_df(df["customer_risk_rating"], "Customer Risk Rating") if "customer_risk_rating" in df.columns else pd.DataFrame()
pep_df         = flag_summary(df["pep_flag"], "PEP Flag", {0:"Non-PEP",1:"PEP"}) if "pep_flag" in df.columns else pd.DataFrame()
industry_df    = pct_df(df["industry"], "Industry") if "industry" in df.columns else pd.DataFrame()
acc_type_df    = pct_df(df["account_type"], "Account Type") if "account_type" in df.columns else pd.DataFrame()

# ── 6. Device Profile ─────────────────────────────────────────────────────────
os_df          = pct_df(df["os_type"], "OS Type") if "os_type" in df.columns else pd.DataFrame()
rooted_df      = flag_summary(df["rooted_flag"], "Rooted", {0:"Not Rooted",1:"Rooted"}) if "rooted_flag" in df.columns else pd.DataFrame()
vpn_df         = flag_summary(df["vpn_flag"], "VPN", {0:"No VPN",1:"VPN Active"}) if "vpn_flag" in df.columns else pd.DataFrame()
emu_df         = flag_summary(df["emulator_flag"], "Emulator", {0:"Not Emulator",1:"Emulator"}) if "emulator_flag" in df.columns else pd.DataFrame()
dev_age_stats  = stats_df(df["device_age_days"], "Device Age (Days)") if "device_age_days" in df.columns else pd.DataFrame()

# ── 7. Beneficiary Profile ────────────────────────────────────────────────────
ben_type_df    = pct_df(df["beneficiary_type"], "Beneficiary Type") if "beneficiary_type" in df.columns else pd.DataFrame()
ben_risk_df    = pct_df(df["beneficiary_country_risk"], "Beneficiary Country Risk") if "beneficiary_country_risk" in df.columns else pd.DataFrame()
high_risk_b    = flag_summary(df["high_risk_beneficiary"], "High Risk Bene", {0:"Normal",1:"High Risk"}) if "high_risk_beneficiary" in df.columns else pd.DataFrame()

# ── 8. Engineered Features ────────────────────────────────────────────────────
round_df       = flag_summary(df["is_round_amount"], "Round Amount", {0:"No",1:"Yes"}) if "is_round_amount" in df.columns else pd.DataFrame()
dormancy_df    = flag_summary(df["dormancy_flag"], "Dormancy", {0:"Active",1:"Dormant"}) if "dormancy_flag" in df.columns else pd.DataFrame()
ratio_stats    = stats_df(df["amount_to_balance_ratio"], "Amount-to-Balance Ratio") if "amount_to_balance_ratio" in df.columns else pd.DataFrame()
log_amt_stats  = stats_df(df["log_amount"], "Log Amount") if "log_amount" in df.columns else pd.DataFrame()

vel_cols_stats = [c for c in [
    "txn_count_last_1hr","txn_count_last_24hr","total_amount_last_24hr",
    "total_amount_last_7d","total_amount_last_30d","txn_velocity_cumulative",
    "cust_txn_count_last_1hr","cust_txn_count_last_24hr",
    "cust_total_amount_last_24hr","cust_total_amount_last_30d","cust_unique_accounts_30d"
] if c in df.columns]
velocity_stats = df[vel_cols_stats].describe().round(2).T.reset_index()
velocity_stats.columns = ["Feature"] + list(velocity_stats.columns[1:])

# ── 9. Rule Engine ────────────────────────────────────────────────────────────
rule_cols = [c for c in df.columns if c.startswith("rule_") and c != "rule_trigger_count"]
rules_df  = rule_summary(rule_cols, df)

trigger_stats = stats_df(df["rule_trigger_count"], "Rule Trigger Count") if "rule_trigger_count" in df.columns else pd.DataFrame()
severity_df   = pct_df(df["max_rule_severity"], "Max Rule Severity") if "max_rule_severity" in df.columns else pd.DataFrame()
wscore_stats  = stats_df(df["weighted_rule_score"], "Weighted Rule Score") if "weighted_rule_score" in df.columns else pd.DataFrame()

# ── Fraud vs Legit rule comparison ───────────────────────────────────────────
if "label" in df.columns and rule_cols:
    fraud_rules = df[df["label"] == 1][rule_cols].mean().round(4)
    legit_rules = df[df["label"] == 0][rule_cols].mean().round(4)
    rule_compare = pd.DataFrame({
        "Rule": [c.replace("rule_","").replace("_"," ").title() for c in rule_cols],
        "Fraud Fire Rate": (fraud_rules.values * 100).round(2),
        "Legit Fire Rate": (legit_rules.values * 100).round(2),
        "Lift (Fraud/Legit)": (fraud_rules.values / (legit_rules.values + 1e-9)).round(2),
    }).sort_values("Lift (Fraud/Legit)", ascending=False).reset_index(drop=True)
else:
    rule_compare = pd.DataFrame()

# ══════════════════════════════════════════════════════════════════════════════
# BUILD EXCEL WORKBOOK
# ══════════════════════════════════════════════════════════════════════════════

wb = Workbook()

# ─── SHEET 1: Dashboard ───────────────────────────────────────────────────────
ws0 = wb.active
ws0.title = "📊 Dashboard"
ws0.sheet_view.showGridLines = False

ws0.merge_cells("A1:F1")
hdr(ws0, 1, 1, "SmartSentry AML — Transaction Data Summary Report", size=15, fill=HDR_FILL)
ws0.row_dimensions[1].height = 40
ws0.merge_cells("A2:F2")
hdr(ws0, 2, 1, f"Total Transactions: {TOTAL:,}  |  Fraud Rate: {fraud_pct}%  |  Unique Customers: {df['customer_id'].nunique():,}", size=10, fill=SUB_FILL)

r = 4
r = write_block(ws0, r, 1, "📌 Key Performance Indicators", overview_kpis)
if not fraud_type_df.empty:
    r = write_block(ws0, r, 1, "🎭 Fraud Type Breakdown", fraud_type_df, flag_high=5.0)

# ─── SHEET 2: Transactions ────────────────────────────────────────────────────
ws1 = wb.create_sheet("💳 Transactions")
ws1.sheet_view.showGridLines = False
ws1.merge_cells("A1:J1")
hdr(ws1, 1, 1, "TRANSACTION CHARACTERISTICS", size=13, fill=HDR_FILL)
ws1.row_dimensions[1].height = 35

r = 3
r = write_block(ws1, r, 1, "Amount Statistics (INR)", amount_stats)
r = write_block(ws1, r, 1, "Channel Distribution", channel_df)
r = write_block(ws1, r, 1, "Transaction Type", txn_type_df)

r2 = 3
r2 = write_block(ws1, r2, 5, "Debit / Credit Split", debit_df)
r2 = write_block(ws1, r2, 5, "Cash Flag", cash_df)
r2 = write_block(ws1, r2, 5, "Round Amount", round_df)
r2 = write_block(ws1, r2, 5, "Dormancy Flag", dormancy_df)

r3 = 3
r3 = write_block(ws1, r3, 9, "Amount-to-Balance Ratio Stats", ratio_stats)
r3 = write_block(ws1, r3, 9, "Log Amount Stats", log_amt_stats)

# ─── SHEET 3: Temporal ────────────────────────────────────────────────────────
ws2 = wb.create_sheet("⏰ Temporal")
ws2.sheet_view.showGridLines = False
ws2.merge_cells("A1:J1")
hdr(ws2, 1, 1, "TEMPORAL DISTRIBUTION", size=13, fill=HDR_FILL)
ws2.row_dimensions[1].height = 35

r = 3
r = write_block(ws2, r, 1, "Hour of Day Distribution", hour_df)
r = write_block(ws2, r, 1, "Day of Week Distribution", dow_df)
r = write_block(ws2, r, 1, "Month Distribution", month_df)

r2 = 3
r2 = write_block(ws2, r2, 5, "Is Night Transaction", night_df)
r2 = write_block(ws2, r2, 5, "Is Weekend Transaction", weekend_df)
r2 = write_block(ws2, r2, 5, "Is Business Hours", biz_df)

# ─── SHEET 4: Customer & Account ─────────────────────────────────────────────
ws3 = wb.create_sheet("👤 Customer & Account")
ws3.sheet_view.showGridLines = False
ws3.merge_cells("A1:J1")
hdr(ws3, 1, 1, "CUSTOMER & ACCOUNT PROFILE", size=13, fill=HDR_FILL)
ws3.row_dimensions[1].height = 35

r = 3
r = write_block(ws3, r, 1, "Occupation", occ_df)
r = write_block(ws3, r, 1, "Industry", industry_df)
r = write_block(ws3, r, 1, "Account Type", acc_type_df)

r2 = 3
r2 = write_block(ws3, r2, 5, "KYC Level", kyc_df)
r2 = write_block(ws3, r2, 5, "Country Risk", crisk_df)
r2 = write_block(ws3, r2, 5, "Customer Risk Rating", cust_risk_df)
r2 = write_block(ws3, r2, 5, "Income Bracket", inc_df)
r2 = write_block(ws3, r2, 5, "PEP Flag", pep_df)

# ─── SHEET 5: Devices ────────────────────────────────────────────────────────
ws4 = wb.create_sheet("📱 Devices")
ws4.sheet_view.showGridLines = False
ws4.merge_cells("A1:J1")
hdr(ws4, 1, 1, "DEVICE PROFILE", size=13, fill=HDR_FILL)
ws4.row_dimensions[1].height = 35

r = 3
r = write_block(ws4, r, 1, "OS Type Distribution", os_df)
r = write_block(ws4, r, 1, "Rooted Device", rooted_df, flag_high=3.0)
r = write_block(ws4, r, 1, "VPN Active", vpn_df, flag_high=5.0)
r = write_block(ws4, r, 1, "Emulator Detected", emu_df, flag_high=2.0)

r2 = 3
r2 = write_block(ws4, r2, 5, "Device Age (Days) Statistics", dev_age_stats)

# ─── SHEET 6: Beneficiaries ───────────────────────────────────────────────────
ws5 = wb.create_sheet("💸 Beneficiaries")
ws5.sheet_view.showGridLines = False
ws5.merge_cells("A1:J1")
hdr(ws5, 1, 1, "BENEFICIARY PROFILE", size=13, fill=HDR_FILL)
ws5.row_dimensions[1].height = 35

r = 3
r = write_block(ws5, r, 1, "Beneficiary Type", ben_type_df)
r = write_block(ws5, r, 1, "Beneficiary Country Risk", ben_risk_df, flag_high=5.0)
r = write_block(ws5, r, 1, "High Risk Beneficiary Flag", high_risk_b, flag_high=5.0)

# ─── SHEET 7: Velocity Features ───────────────────────────────────────────────
ws6 = wb.create_sheet("⚡ Velocity Features")
ws6.sheet_view.showGridLines = False
ws6.merge_cells("A1:K1")
hdr(ws6, 1, 1, "VELOCITY & ROLLING WINDOW FEATURE STATISTICS", size=13, fill=HDR_FILL)
ws6.row_dimensions[1].height = 35

r = 3
if not velocity_stats.empty:
    r = write_block(ws6, r, 1, "Account & Customer Velocity Feature Statistics", velocity_stats)

# ─── SHEET 8: Rule Engine ─────────────────────────────────────────────────────
ws7 = wb.create_sheet("🔴 Rule Engine")
ws7.sheet_view.showGridLines = False
ws7.merge_cells("A1:J1")
hdr(ws7, 1, 1, "AML RULE ENGINE — TRIGGER ANALYSIS", size=13, fill=HDR_FILL)
ws7.row_dimensions[1].height = 35

r = 3
if not rules_df.empty:
    r = write_block(ws7, r, 1, "All Rules — Trigger Frequency (sorted by volume)", rules_df)

r2 = 3
if not trigger_stats.empty:
    r2 = write_block(ws7, r2, 6, "Rule Trigger Count Statistics", trigger_stats)
if not severity_df.empty:
    r2 = write_block(ws7, r2, 6, "Max Rule Severity Distribution", severity_df)
if not wscore_stats.empty:
    r2 = write_block(ws7, r2, 6, "Weighted Rule Score Statistics", wscore_stats)

# ─── SHEET 9: Rule Lift (Fraud vs Legit) ──────────────────────────────────────
if not rule_compare.empty:
    ws8 = wb.create_sheet("📈 Rule Lift Analysis")
    ws8.sheet_view.showGridLines = False
    ws8.merge_cells("A1:E1")
    hdr(ws8, 1, 1, "RULE LIFT ANALYSIS — Fraud Fire Rate vs Legitimate Fire Rate", size=13, fill=HDR_FILL)
    ws8.merge_cells("A2:E2")
    hdr(ws8, 2, 1, "Lift > 1 means rule fires more on fraud. Higher lift = stronger discriminating power.", size=9, fill=SUB_FILL)
    ws8.row_dimensions[1].height = 35
    write_block(ws8, 4, 1, "Rule Discrimination Power (sorted by Lift)", rule_compare)

# ── Save ───────────────────────────────────────────────────────────────────────
wb.save(OUTPUT_PATH)
print(f"\n✅ Summary saved to: {OUTPUT_PATH}")
print(f"   Sheets: {[ws.title for ws in wb.worksheets]}")


✅ Summary saved to: c:\Users\VISHNUPRIYA\OneDrive\Desktop\Freelancing\AIGEN\smartsentry_aml_model\transactions_summary.xlsx
   Sheets: ['📊 Dashboard', '💳 Transactions', '⏰ Temporal', '👤 Customer & Account', '📱 Devices', '💸 Beneficiaries', '⚡ Velocity Features', '🔴 Rule Engine', '📈 Rule Lift Analysis']
